# **Studio-Afrik**

## Overview

## Business Understanding

## Data Understanding

## Imports & Data

The code cell below contain libraries that are essential in this project analysis.

In [113]:
# Perfom data manipulation and analysis.
import pandas as pd

# Perfoming mathematical calculations.
import numpy as np

# The two libraries below will aid in creating visualizations.
import matplotlib as plt 
import seaborn as sns

# This library helps in accessing our relational database.
import sqlite3

# Code below imports all code in the custom_func file
from custom_code import *

### Working with available data

I. **Relational Database**

1. im.db

II. **CSV FILES**

2. tn.movie_budgets.csv

### 1. IMDB

This dataset comprises of multiple tables containing information about movies. The tables of interest are: movie_basics and movie_ratings.

The movie_basics table includes movie titles, release year, and genres. The movie_ratings table includes average movie rating and number of votes. The <br/>
primary key for both tables is movie_id which will help in joining the two tables.

Here, I am creating a Connection to the relational database from im.db using module sqlite3.

In [114]:
path = "Data/im.db"
conn = sqlite3.connect(path)

Display all the tables in the imdb database

In [115]:
query = """
SELECT name 
FROM sqlite_master 
    WHERE type = 'table';
"""
# print tables in the sql database
imdb_tables = pd.read_sql(query, conn) 

In order to start using our data, you will have to view information from tables I find relevant to complete this analysis.

Movie Ratings table

In [116]:
query = """
SELECT * 
FROM movie_ratings;
"""

movie_ratings = pd.read_sql(query, conn)
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


Movie Basics table

In [117]:
query = """
SELECT * 
FROM movie_basics;
"""

movie_basics= pd.read_sql(query, conn)
movie_basics.head(5)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


Director Names

JOIN persons table and directors table

In [118]:
query = """
SELECT DISTINCT d.movie_id, d.person_id AS director_id, p.primary_name AS director_name
FROM persons AS p
    INNER JOIN directors AS d
        USING(person_id);
"""

director_data = pd.read_sql(query, conn)
director_data

,movie_id,director_id,director_name
0,tt1592569,nm0062879,Ruel S. Bayani
1,tt2057445,nm0062879,Ruel S. Bayani
2,tt2590280,nm0062879,Ruel S. Bayani
3,tt8421806,nm0062879,Ruel S. Bayani
4,tt3501180,nm0064023,Bryan Beasley
...,...,...,...
163528,tt8697720,nm9971456,Zheng Wei
163529,tt8715016,nm9980896,Rama Narayanan
163530,tt8919136,nm9980896,Rama Narayanan
163531,tt8717234,nm9981679,Samir Eshra


### 2. tn.movie_budgets.csv

This dataset contain financial information about each movie in their dataset. The columns production budget, domestic gross and worldwide gross <br/>
describes how much was spent during production and its return after production in each movie.

It will also help us calculate the foreign gross and net profit based on domestic,foreign and total profit.

In [119]:
finance_df = pd.read_csv("Data/tn.movie_budgets.csv")
display(finance_df.head())
# Check if our dataset contains missing values
display(finance_df.info())

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,18-Dec-09,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,20-May-11,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,7-Jun-19,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,1-May-15,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,15-Dec-17,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


None

## Data Preparation

Transforming raw data from the above datasets into a format that can be easily and effectively used for analysis.

I will Pick the movie basics table and merge it with the movie ratings table form imdb to get more detailed information about movies.

In [123]:
movie_details = movie_basics.merge(movie_ratings, how="inner", left_on="movie_id", right_on="movie_id")

movie_details

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119
...,...,...,...,...,...,...,...,...
73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6
73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136
73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8
73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5


In [124]:
display(movie_ratings.info())
display(movie_basics.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


None

Check for duplicates in column director_id and person_id which are unique identifiers

In [ ]:
# Display count of unique and duplicated data
director_name = ["director_id", "person_id"]
# imported function from custom_func.py.
dup(director_data, director_name)

Remove duplicates together with any related data from entire dataframe above.

In [ ]:
director_data.drop_duplicates(subset='director_name', ignore_index=True, inplace=True)

# Confirm if the duplicates were removed and the total number of rows matches the above false value derived from value_counts
director_data

,director_id,person_id,director_name,primary_profession
0,nm0062879,nm0062879,Ruel S. Bayani,"director,production_manager,miscellaneous"
1,nm0064023,nm0064023,Bryan Beasley,"director,producer,writer"
2,nm0066268,nm0066268,Steve Mitchell Beebe,"editorial_department,director,editor"
3,nm0067234,nm0067234,Hans Beimler,"producer,writer,miscellaneous"
4,nm0068874,nm0068874,Hava Kohav Beller,"director,writer,producer"
...,...,...,...,...
106752,nm9951412,nm9951412,Dale Bailey,director
106753,nm9959513,nm9959513,Irem Yavuz,"director,editorial_department"
106754,nm9980896,nm9980896,Rama Narayanan,"director,writer"
106755,nm9981679,nm9981679,Samir Eshra,"director,writer,cinematographer"


### Create a dataframe below : **movie_data** 

**Join movie_basics, movie_ratings, directors**

Display Genre, Average ratings with respect to the directors.

In [ ]:
movie_data = pd.read_sql("""
SELECT DISTINCT mb.movie_id, mb.primary_title, mb.start_year, mb.runtime_minutes AS duration, mb.genres, 
                mr.averagerating, mr.numvotes, d.person_id AS director_id, p.primary_name AS director_name, p.primary_profession
FROM movie_basics AS mb
    JOIN movie_ratings AS mr
        on mr.movie_id = mb.movie_id
    JOIN directors AS d
        on d.movie_id = mb.movie_id
    JOIN persons AS p
        on p.person_id = d.person_id                 
    ;
""", 
conn)

movie_data

,movie_id,primary_title,start_year,duration,genres,averagerating,numvotes,director_id,director_name,primary_profession
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,nm0712540,Harnam Singh Rawail,"director,writer,producer"
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43,nm0002411,Mani Kaul,"director,writer,actor"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,nm0000080,Orson Welles,"actor,director,writer"
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,nm0611531,Hrishikesh Mukherjee,"director,editor,writer"
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,nm0749914,Raoul Ruiz,"director,writer,producer"
...,...,...,...,...,...,...,...,...,...,...
86025,tt9913084,Diabolik sono io,2019,75.0,Documentary,6.2,6,nm0812850,Giancarlo Soldi,"director,writer,producer"
86026,tt9914286,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,nm4394529,Ahmet Faik Akinci,"director,writer"
86027,tt9914642,Albatross,2017,NaN,Documentary,8.5,8,nm5300859,Chris Jordan,"director,writer,editor"
86028,tt9914942,La vida sense la Sara Amat,2019,NaN,None,6.6,5,nm1716653,Laura Jou,"miscellaneous,actress,director"


In [ ]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86030 entries, 0 to 86029
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            86030 non-null  object 
 1   primary_title       86030 non-null  object 
 2   start_year          86030 non-null  int64  
 3   duration            77112 non-null  float64
 4   genres              85232 non-null  object 
 5   averagerating       86030 non-null  float64
 6   numvotes            86030 non-null  int64  
 7   director_id         86030 non-null  object 
 8   director_name       86030 non-null  object 
 9   primary_profession  85941 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 6.6+ MB


Check for duplicates in columns movie_id and primary_title from the movie_data dataframe above which identifies as sensitive data. 

In [ ]:
movie_data_columns = ["movie_id", "primary_title"]

# imported function from custom_func.py.
dup(movie_data, movie_data_columns)

Checking for Duplicates

=> movie_id
False    73103
True     12927
Name: movie_id, dtype: int64
 ***Found duplicates
------------------

=> primary_title
False    69285
True     16745
Name: primary_title, dtype: int64
 ***Found duplicates
------------------



**Delete duplicates** in the specified columns together with any related data in the dataframe


In [ ]:
# remove_dup from column primary_title as observed above.
movie_data.drop_duplicates(subset=movie_data_columns, ignore_index=True, inplace=True)

In [ ]:
#Display all information in the movie_data dataframe.
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73103 entries, 0 to 73102
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            73103 non-null  object 
 1   primary_title       73103 non-null  object 
 2   start_year          73103 non-null  int64  
 3   duration            65760 non-null  float64
 4   genres              72440 non-null  object 
 5   averagerating       73103 non-null  float64
 6   numvotes            73103 non-null  int64  
 7   director_id         73103 non-null  object 
 8   director_name       73103 non-null  object 
 9   primary_profession  73036 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 5.6+ MB


**Dropping missing values** and all data that is associated with columns duration and genres.

In [ ]:
movie_data.dropna(subset=["duration", "genres"], inplace=True)

In [ ]:
# View if our columns have identical number of rows and entries in the dataframe.
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65304 entries, 0 to 73102
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            65304 non-null  object 
 1   primary_title       65304 non-null  object 
 2   start_year          65304 non-null  int64  
 3   duration            65304 non-null  float64
 4   genres              65304 non-null  object 
 5   averagerating       65304 non-null  float64
 6   numvotes            65304 non-null  int64  
 7   director_id         65304 non-null  object 
 8   director_name       65304 non-null  object 
 9   primary_profession  65248 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 5.5+ MB


In [ ]:
movie_data.tail(20)

,movie_id,primary_title,start_year,duration,genres,averagerating,numvotes,director_id,director_name,primary_profession
73080,tt9900180,Aavahayami,2017,97.0,Mystery,9.0,9,nm10531566,Girish Kumar,"director,actor,manager"
73081,tt9900388,Hansa Ek Sanyog,2019,135.0,Drama,7.8,8,nm10531592,Dhiraj Verma,None
73082,tt9900552,The Sacred Riana: Beginning,2019,111.0,Horror,7.2,17,nm4479905,Billy Christian,"director,writer,assistant_director"
73083,tt9903716,Jessie,2019,106.0,"Horror,Thriller",8.5,100,nm10532693,Aswani Kumar V.,"director,writer"
73084,tt9903952,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5,nm10532891,Loco Meisenkaiser,"director,actor,writer"
73085,tt9904014,Lost in Klessin,2018,90.0,War,7.3,12,nm6960704,Heintje Peter,"director,editor,producer"
73086,tt9904820,American Terror Story,2019,76.0,Horror,2.6,19,nm4677501,Terry Spears,"sound_department,producer,director"
73087,tt9904844,Ott Tänak: The Movie,2019,125.0,Documentary,8.7,218,nm4942142,Tarvo Mölder,"director,actor"
73088,tt9905412,Ottam,2019,120.0,Drama,8.1,505,nm10533890,Zam,director
73089,tt9905462,Pengalila,2019,111.0,Drama,8.4,600,nm0151535,T.V. Chandran,"director,writer,actor"


In [ ]:
movie_data["director_name"].value_counts()

Sergey A.              33
Michael Fredianelli    28
Nayato Fio Nuala       28
David DeCoteau         26
Ram Gopal Varma        21
                       ..
Andreas Pfiffner        1
Stephen van Vuuren      1
Bruce Wodder            1
Aldo Lado               1
Ekoué Labitey           1
Name: director_name, Length: 44032, dtype: int64

In [ ]:
# check for typos
for x in movie_data["director_name"]:
    if x == "None":
        print(x)
        print("")
    else:
       pass